In [106]:
import numpy as np
import pandas as pd
import pymongo
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# all imports for step 5
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

# !pip install torchsample
# from torchsample.modules import ModuleTrainer



from sklearn import preprocessing
import pdb
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gzip
import matplotlib.pyplot as plt
import os
# !pip install torchsample
# from torchsample.modules import ModuleTrainer

from sklearn import preprocessing
import numpy as np
import torch.nn as nn
from torch.nn.utils.rnn import (
    pack_padded_sequence, pad_packed_sequence)


In [110]:
class OrderbookDataset(Dataset):
    def __init__(self,percentage):
        self.percentage = percentage
        
        client = pymongo.MongoClient('localhost',27017)
        db = client['shrimpy_binance_btc_usd']
        self.collection = db['entries']
        all_length = self.collection.find().count()
        self.percentage = (all_length - 32000)/all_length
        
        self.real_length = int(all_length*percentage) + 16
        self.first_time = self.collection.find_one()
        self.first_time = self.first_time['_id'] 
        self.extra_time = 0
    
    def __getitem__(self, item):
        try:
            ret = self.collection.find_one({'_id':self.first_time+60*item+self.extra_time})
            data = ret['data']
            label = ret['label']
            data = np.array(data)
            if label == 1:
                label = np.array(0)
            elif label == -1:
                label = np.array(2)
            else:
                label = np.array(1)
            return data,label
        except:
            self.extra_time += 60
            self.get_item(item)
        
    def get_item(self, item):
        try:
            ret = self.collection.find_one({'_id':self.first_time+60*item+self.extra_time})
            data = ret['data']
            label = ret['label']
            data = np.array(data)
            if label == 1:
                label = np.array(0)
            elif label == -1:
                label = np.array(2)
            else:
                label = np.array(1)
            return data,label
        except:
            self.extra_time += 60
            self.get_item(item)
    
    def __len__(self):
        return self.real_length

In [111]:
class OrderbookTestDataset(Dataset):
    def __init__(self,percentage):
        self.percentage = percentage
        client = pymongo.MongoClient('localhost',27017)
        db = client['shrimpy_binance_btc_usd']
        self.collection = db['entries']
        all_length = self.collection.find().count()
        self.real_length = 32000
        self.real_length = int(self.real_length) if self.real_length == int(self.real_length) else int(self.real_length) + 1
        self.first_time = self.collection.find_one()
        self.first_time = self.first_time['_id']
        self.first_time = self.collection.find_one({'_id':self.first_time+(all_length-self.real_length)*60})
        self.first_time = self.first_time['_id']
        
    def __getitem__(self, item):
        ret = self.collection.find_one({'_id':self.first_time+60*item})
        data = ret['data']
        label = ret['label']
        data = np.array(data)
        if label == 1:
            label = np.array(0)
        elif label == -1:
            label = np.array(2)
        else:
            label = np.array(1)
        return data,label
    
    def __len__(self):
        return self.real_length
        
    

In [112]:
train = OrderbookDataset(0.9)
test = OrderbookTestDataset(0.9)


/home/nikopi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  
/home/nikopi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  import sys


In [122]:
class PadPackedSequence(nn.Module):
    """Some Information about PadPackedSequence"""
    def __init__(self, batch_first=True):
        super(PadPackedSequence, self).__init__()
        self.batch_first = batch_first

    def forward(self, x, lengths):
        max_length = lengths.max().item()
        x, _ = pad_packed_sequence(
            x, batch_first=self.batch_first, total_length=max_length)
        return x


class PackSequence(nn.Module):
    def __init__(self, batch_first=True):
        super(PackSequence, self).__init__()
        self.batch_first = batch_first

    def forward(self, x, lengths):
        x = pack_padded_sequence(
            x, lengths,
            batch_first=self.batch_first,
            enforce_sorted=False)
        lengths = lengths[x.sorted_indices]
        return x, lengths

In [123]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=True,
                 layers=1, bidirectional=False, merge_bi='cat', dropout=0,
                 rnn_type='lstm', packed_sequence=True, device='cpu'):

        super(RNN, self).__init__()
        self.device = device
        self.bidirectional = bidirectional
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        self.merge_bi = merge_bi
        self.rnn_type = rnn_type.lower()

        rnn_cls = nn.LSTM if self.rnn_type == 'lstm' else nn.GRU
        self.rnn = rnn_cls(input_size,
                           hidden_size,
                           batch_first=batch_first,
                           num_layers=layers,
                           bidirectional=bidirectional)
        self.drop = nn.Dropout(dropout)
        self.packed_sequence = packed_sequence
        if packed_sequence:
            self.pack = PackSequence(batch_first=batch_first)
            self.unpack = PadPackedSequence(batch_first=batch_first)

    def _merge_bi(self, forward, backward):
        if self.merge_bi == 'sum':
            return forward + backward
        return torch.cat((forward, backward), dim=-1)

    def _select_last_unpadded(self, out, lengths):
        gather_dim = 1 if self.batch_first else 0
        gather_idx = ((lengths - 1)  # -1 to convert to indices
                      .unsqueeze(1)  # (B) -> (B, 1)
                      .expand((-1, self.hidden_size))  # (B, 1) -> (B, H)
                      # (B, 1, H) if batch_first else (1, B, H)
                      .unsqueeze(gather_dim))
        # Last forward for real length or seq (unpadded tokens)
        last_out = out.gather(gather_dim, gather_idx).squeeze(gather_dim)
        return last_out

    def _final_output(self, out, lengths):
        # Collect last hidden state
        # Code adapted from https://stackoverflow.com/a/50950188
        if not self.bidirectional:
            return self._select_last_unpadded(out, lengths)

        forward, backward = (out[..., :self.hidden_size],
                             out[..., self.hidden_size:])
        # Last backward corresponds to first token
        last_backward_out = (backward[:, 0, :]
                             if self.batch_first
                             else backward[0, ...])
        # Last forward for real length or seq (unpadded tokens)
        last_forward_out = self._select_last_unpadded(forward, lengths)
        return self._merge_bi(last_forward_out, last_backward_out)

    def forward(self, x, lengths):
        self.rnn.flatten_parameters()
        if self.packed_sequence:
            x, lengths = self.pack(x, lengths)
        out, hidden = self.rnn(x)
        if self.packed_sequence:
            out = self.unpack(out, lengths)
        out = self.drop(out)

        last_timestep = self._final_output(out, lengths)
        return out, last_timestep, hidden


In [186]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size = [2,1], stride = [2,1],padding=0),
            nn.LeakyReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,16,kernel_size = (2,1), stride = (2,1),padding=0),
            nn.LeakyReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16,16,kernel_size = (10,1), stride = 1),
            nn.LeakyReLU()
        )
        
        # /////////////////////////INCEPTION////////////////////
        self.layer11 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size = (1,1), stride = 1,padding=0),
            nn.LeakyReLU()
        )
        self.layer12 = nn.Sequential(
            
            nn.Conv2d(32,32,kernel_size = (1,3), stride = 1,padding=0),
            nn.LeakyReLU()
        )
        self.layer21 = nn.Sequential(
            
            nn.Conv2d(16,32,kernel_size = (1,1), stride = 1,padding=0),
            nn.LeakyReLU()
        )
        self.layer22 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size = (1,5), stride = 1,padding=0),
            nn.LeakyReLU()
        )
        self.layer31 = nn.Sequential(
            nn.MaxPool2d(kernel_size=(1,3),stride=1)
        )
        self.layer32 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size = (1,1), stride = 1,padding=0),
            nn.LeakyReLU()
        )
        #/////////////////////////LSTM/////////////////////
        self.lstm = RNN(32,32,packed_sequence=False)
#        nn.LSTM(32,30,1,batch_first=True)
        #////////////////////////LINEAR///////////////////
        self.linear = nn.Linear(32,3)
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        x = out
        out1 = self.layer11(x)
        out1 = self.layer12(out1)
        
        out2 = self.layer21(x)
        out2 = self.layer22(out2)
        
        out3 = self.layer31(x)
        out3 = self.layer32(out3)
        
        out = torch.cat((out1,out2),-1)
        out = torch.cat((out,out3),-1)
        out = out.squeeze(2)
        out = out.transpose(1,2)
        
        #print(out.shape)
        #hidden_b = torch.zeros(1,out.shape[0],30).double()
        #self.hiddens = (hidden_b, hidden_b) 
        _, lstm_out, self.hiddens = self.lstm.forward(out, torch.tensor([22,22,22,22,22,22,22,22,
                                                                        22,22,22,22,22,22,22,22,
                                                                        22,22,22,22,22,22,22,22,
                                                                        22,22,22,22,22,22,22,22,
                                                                        ]))
        
        #correctout = lstm_out.reshape(32,-1)
#         #if correctout.shape[1] < 660:
#             print('oops')
#             needpading = 660-correctout.shape[1]
#             padleft = int(needpading / 2)
#             padright = int(needpading/2)
#             if needpading != padleft+padright:
#                 padright+= 1
#             correctout = F.pad(correctout,(int(padleft),int(padright)))
            
        lastout = self.linear(lstm_out)
        
        return lastout

    

In [189]:

output_dim = 1
epochs = 30
BATCH_SZ = 32

dl_train = DataLoader(train, batch_size = BATCH_SZ, shuffle=False)
dl_test =  DataLoader(test, batch_size = BATCH_SZ, shuffle=False)

cnn = Model().double()

optimizer = optim.Adam(cnn.parameters(), lr = 0.01,eps=0.5)
criterion = nn.CrossEntropyLoss()

In [190]:

min_loss = 10000000
train_running_average_loss = np.zeros((epochs))
val_running_average_loss = np.zeros((epochs))
learning = np.zeros((epochs))
for epoch in range(epochs):
    print('epoch: ',epoch,' out of ',epochs)
    
    for i,data in enumerate (dl_train):
        X_batch, y_batch = data
        
        X_batch.unsqueeze_(-1)
        X_batch = X_batch.transpose(1,3)
        #X_batch = X_batch.transpose(-1,1)
        X_batch = X_batch.double()
        
        optimizer.zero_grad()
        out = cnn(X_batch)
    
        loss = criterion(out, y_batch.long())
        loss.backward()
        optimizer.step()
        train_running_average_loss[epoch] += loss.detach().item()
    print(train_running_average_loss)
    train_running_average_loss[epoch] =  train_running_average_loss[epoch]/i
        ###############################################    
    # Now i ll run the validation set
    for i, data in enumerate(dl_test):
        X_batch, y_batch = data
        
        X_batch.unsqueeze_(-1)
        X_batch = X_batch.transpose(1,3)
        #X_batch = X_batch.transpose(-1,1)
        X_batch = X_batch.double()
        
        out  = cnn(X_batch)
        loss = criterion(out, y_batch.long())
        
        val_running_average_loss[epoch] += loss.detach().item()
    val_running_average_loss[epoch] =  val_running_average_loss[epoch]/i
    if val_running_average_loss[epoch] < min_loss:
        min_loss = val_running_average_loss[epoch]
        max_model = cnn
    print('For Epoch:', epoch, 'train loss:', train_running_average_loss[epoch], 'val loss:', val_running_average_loss[epoch])
    if epoch >1:
        if 0.9*val_running_average_loss[epoch] > val_running_average_loss[epoch-1] :
            break
plt.figure()        
plt.plot(train_running_average_loss, 'b', val_running_average_loss, 'g', epochs) 
plt.xlabel('epochs')
plt.ylabel('train_loss blue, val_loss green')
plt.show()



epoch:  0  out of  30
[1832.10157688    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.        ]
For Epoch: 0 train loss: 0.18841028145627386 val loss: 0.2340269831193431
epoch:  1  out of  30
[1.88410281e-01 1.27101357e+03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
For Epoch: 1 train loss: 0.130708923

[1.88410281e-01 1.30708923e-01 1.27868073e-01 1.26530638e-01
 1.25504797e-01 1.24664936e-01 1.23934723e-01 1.23299878e-01
 1.22661237e-01 1.22009222e-01 1.21427669e-01 1.20807826e-01
 1.20158620e-01 1.19592225e-01 1.19108929e-01 1.15413264e+03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
For Epoch: 15 train loss: 0.11868908219484249 val loss: 0.21957540009709367
epoch:  16  out of  30
[1.88410281e-01 1.30708923e-01 1.27868073e-01 1.26530638e-01
 1.25504797e-01 1.24664936e-01 1.23934723e-01 1.23299878e-01
 1.22661237e-01 1.22009222e-01 1.21427669e-01 1.20807826e-01
 1.20158620e-01 1.19592225e-01 1.19108929e-01 1.18689082e-01
 1.15055622e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00

TypeError: zip argument #5 must support iteration

In [191]:
print(max_model)

Model(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=[2, 1], stride=[2, 1])
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(2, 1), stride=(2, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 16, kernel_size=(10, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer11): Sequential(
    (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer12): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer21): Sequential(
    (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer22): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 5), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer31): Sequential(
    (0): MaxPool2d(kernel_size=(1, 3), stride=1, padding=0, dilation=1, cei